In [18]:
# Import needed packages for analysis
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib
from scipy.stats import pearsonr
from scipy.stats import kendalltau

import cfl.util.brain_util as BU
import cfl.util.brain_vis as BV
import cfl.util.fear_mice_functions as fm 

# load response data 
Y = pd.read_pickle('Y.pkl')

behav_csv = 'PTSD_Data_Share\Behavior_data\PTSD_PerLight.csv'

mri_dir, mri_dims, affine, dir_labels = fm.get_global_values()

# load all the images in RPS orientation 
X, Y_unused = BU.load_data(mri_dir, behav_csv, mri_dims, ori='RPS')

In [5]:
data_id = 'X_'

# Correlation between each voxel of brains and Y (percent time spent in the light)

For each voxel: 
- Have 20 pairs of values: MRI (fear - pre-fear)  and Y (fear- pre-fear)
- find correlation between these pairs for ^ for each voxel 
- map that back onto the brain 

## Calculate the X diffs 

In [38]:
indices_dir = fm.geno_time_indices_dir(Y)

X_diffs_KO = X[indices_dir['Fear_KO']] - X[indices_dir['PreF_KO']]
X_diffs_WT = X[indices_dir['Fear_WT']] - X[indices_dir['PreF_WT']]
X_diffs_all = np.vstack((X_diffs_KO, X_diffs_WT)) # for both genotypes

## Calculate the Y diffs

In [30]:
indices_dir = fm.geno_time_indices_dir(Y)

Y_diffs_KO = Y.PerLight[indices_dir['Fear_KO']].to_numpy() - Y.PerLight[indices_dir['PreF_KO']].to_numpy()
Y_diffs_WT = Y.PerLight[indices_dir['Fear_WT']].to_numpy() - Y.PerLight[indices_dir['PreF_WT']].to_numpy()
Y_diffs_all = np.hstack((Y_diffs_KO, Y_diffs_WT)) # for both genotypes

## Compute the Pearson correlations

In [41]:
def calculate_correlations_and_save(X_diffs, Y_diffs, save_name, corr_method, mri_dims, data_id, affine): 
    # correlations
    corrs = np.zeros(np.prod(mri_dims))
    for voxel in range(np.prod(mri_dims)): 
        value = corr_method(Y_diffs, X_diffs[:, voxel])[0]
        if np.isnan(value): #if x is all 0s, pearson returns a nan
            value = 0 
        corrs[voxel] = value

    # save 
    np.save(os.path.join("npy_files", data_id + save_name), corrs)
    fm.save_as_nifti(corrs, os.path.join('correctly_oriented_niis', data_id + save_name), mri_dims, affine)
    return corrs

In [46]:
calculate_correlations_and_save(X_diffs_all, Y_diffs_all, 'corr_pearson', pearsonr, mri_dims, data_id, affine)
calculate_correlations_and_save(X_diffs_KO, Y_diffs_KO, 'corr_pearson_KO', pearsonr, mri_dims, data_id, affine)
calculate_correlations_and_save(X_diffs_WT, Y_diffs_WT, 'corr_pearson_WT', pearsonr, mri_dims, data_id, affine)

array([0., 0., 0., ..., 0., 0., 0.])

## Compute the Kendall Correlations 

In [45]:
calculate_correlations_and_save(X_diffs_all, Y_diffs_all, 'corr_kendall', kendalltau, mri_dims, data_id, affine)
calculate_correlations_and_save(X_diffs_KO, Y_diffs_KO, 'corr_kendall_KO', kendalltau, mri_dims, data_id, affine)
calculate_correlations_and_save(X_diffs_WT, Y_diffs_WT, 'corr_kendall_WT', kendalltau, mri_dims, data_id, affine)

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
import matplotlib.lines as mlines
plt.scatter(all_data['X_diffs'], all_data['Y_diffs'])

m, b = np.polyfit(all_data['X_diffs'], all_data['Y_diffs'],  1)

#add linear regression line to scatterplot 
plt.plot(all_data['X_diffs'], m*all_data['X_diffs']+b)

plt.ylabel('Change in time spent in light')
plt.xlabel('Change in MRI activity')
plt.suptitle('X and Y Diffs for voxel at (26, 137, 37')
plt.show()